# Scraping articles 
source: https://femminicidioitalia.info/lista/2022


In [1]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time 
import os 
import wget 
import getpass
import pandas as pd

In [2]:
s= Service('\Applications\chromedriver')

How to set up selenium : https://www.youtube.com/watch?v=95OqqfGi2Eg

In [233]:
#Optional- set options
chromeOptions = Options()
chromeOptions.headless= False 

#initialize driver 

driver= webdriver.Chrome(service=s, options= chromeOptions)

/var/folders/27/q_rl13_54jg1vgws9ntkdmbr0000gn/T/ipykernel_90301/3033477291.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chromeOptions.headless= False


In [234]:
def open_website(): 
    '''
    opens website and clicks on cookie banner
    '''
    driver.get("https://femminicidioitalia.info/")
    print("opened")
    #cookie banner 
    time.sleep(5)
    cookie1=driver.find_element(By.XPATH, "//button[@class='fc-button fc-cta-consent fc-primary-button']")
    cookie1.click()

open_website()
    

opened


In [235]:
#try for 2020 : this opens the right list
def find_victims(year): 
    year.click()
    #find all victims 
    victims=driver.find_elements(By.CLASS_NAME, "victimName")
    #check if it worked 
    print(type(victims))
    nvic=len(victims)
    print(f"in 2020 there are articles about {nvic} victims.")

year20=driver.find_element(By.XPATH, "//a[@title='Lista dei femminicidi in Italia nel 2020']")

find_victims(year20)

victims[0].click()



<class 'list'>
in 2020 there are articles about 64 victims.


InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000102ddefa8 chromedriver + 4886440
1   chromedriver                        0x0000000102d5c643 chromedriver + 4351555
2   chromedriver                        0x00000001029aa99d chromedriver + 477597
3   chromedriver                        0x00000001029dec1f chromedriver + 691231
4   chromedriver                        0x0000000102a154d6 chromedriver + 914646
5   chromedriver                        0x0000000102a113bd chromedriver + 897981
6   chromedriver                        0x0000000102a106df chromedriver + 894687
7   chromedriver                        0x0000000102974b6a chromedriver + 256874
8   chromedriver                        0x0000000102dabb9e chromedriver + 4676510
9   chromedriver                        0x0000000102db091e chromedriver + 4696350
10  chromedriver                        0x0000000102db819f chromedriver + 4727199
11  chromedriver                     

In [232]:
#create dataframe 
df1= pd.DataFrame(columns=('victim', 'multiple', 'killer', 'presumed', 'location', 'death_loc', 'province', 'date', 'links' ))
df1

,victim,multiple,killer,presumed,location,death_loc,province,date,links


In [229]:
#test for 1 
test=victims[0]
test.click()
time.sleep(5)
#el=driver.find_elements(By.XPATH, "//span[@class='recapvalue fontspecialelite']")

#driver.back()
#works 

InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000102ddefa8 chromedriver + 4886440
1   chromedriver                        0x0000000102d5c643 chromedriver + 4351555
2   chromedriver                        0x00000001029aa99d chromedriver + 477597
3   chromedriver                        0x00000001029dec1f chromedriver + 691231
4   chromedriver                        0x0000000102a154d6 chromedriver + 914646
5   chromedriver                        0x0000000102a113bd chromedriver + 897981
6   chromedriver                        0x0000000102a106df chromedriver + 894687
7   chromedriver                        0x0000000102974b6a chromedriver + 256874
8   chromedriver                        0x0000000102dabb9e chromedriver + 4676510
9   chromedriver                        0x0000000102db091e chromedriver + 4696350
10  chromedriver                        0x0000000102db819f chromedriver + 4727199
11  chromedriver                     

In [215]:
def victim_info(): 
    '''
    on the page of the victim takes information and links
    '''
    ti=driver.find_elements(By.XPATH, "//span[@class='recaplabel fontspecialelite alignright']")
    el=driver.find_elements(By.XPATH, "//span[@class='recapvalue fontspecialelite']")

    for t, e in zip(ti, el):
        #print(t.text, e.text)
        if t.text=="Vittima:":
            df1['victim']=[e.text]
            df1['multiple']=0
        elif t.text=="Vittima:":
            df1['victim']=[e.text]
            df1['multiple']=1
        elif t.text=="Presunto killer:":
            df1['killer']=[e.text]
            df1['presumed']=1
        elif t.text=="Killer:":
            df1['killer']=[e.text]
            df1['presumed']=0
        elif t.text=="Località:":
            df1['location']=[e.text]
        elif t.text=="Provincia": 
            df1['province']=[e.text]
        elif t.text=="Decesso:":
            df1['death_loc']=[e.text]
        elif t.text=="Data:":
            df1['date']=[e.text]
        else: 
            print("Missing info:", e.text)
    #print(df1)

                  

In [209]:
def newspaper_info():
    #get links
    hr=driver.find_elements(By.XPATH, "//a[@class='fem_sharesource fem_sourceMain']")
    links = [h.get_attribute('href') for h in hr]

    #get titles
    titles =[]
    for h in hr:
        titles.append(h.text)

    #get name of newspaper
    np=driver.find_elements(By.XPATH, "//cite")
    nps=[]
    for n in np:
        nps.append(n.text)
    
    
    info=[]
    for l, t, n in zip(links, titles, nps):
        li=[]
        li.append(l)
        li.append(t)
        li.append(n)
        info.append(li)
        #print (info)
    df1['links']=[info]
    

In [210]:
for v in victims[0:2]:
    v.click()
    time.sleep(5)
    victim_info()
    newspaper_info()
    df1
    driver.back()
    time.sleep(5)

InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000102ddefa8 chromedriver + 4886440
1   chromedriver                        0x0000000102d5c643 chromedriver + 4351555
2   chromedriver                        0x00000001029aa99d chromedriver + 477597
3   chromedriver                        0x00000001029dec1f chromedriver + 691231
4   chromedriver                        0x0000000102a154d6 chromedriver + 914646
5   chromedriver                        0x0000000102a113bd chromedriver + 897981
6   chromedriver                        0x0000000102a106df chromedriver + 894687
7   chromedriver                        0x0000000102974b6a chromedriver + 256874
8   chromedriver                        0x0000000102dabb9e chromedriver + 4676510
9   chromedriver                        0x0000000102db091e chromedriver + 4696350
10  chromedriver                        0x0000000102db819f chromedriver + 4727199
11  chromedriver                     